In [4]:
# Python libraries
import cv2
import time
import matplotlib.pyplot as plt

# I used a python library called deepface for emotion detection
from deepface import DeepFace
from gtts import gTTS
from threading import Timer

# This function's gonna create an audio file where you will here your emotion:
def audio(result):
    import os
    audio = gTTS(
            text="You are "+result, 
            lang='en', slow=False
            )
    # I'll save the audio file in a directory that I named it '/result'
    filename = f"{time.time()}.mp3"
    audio.save(f"results/{filename}")
    os.system(f'start results/{filename}')


cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX
last_time = time.time()
diffrence_happend = False
last_result = ""


while True:
    # Taking frames
    ret, frame = cap.read()
    result = DeepFace.analyze(frame, actions = ['emotion'])
    result = result['dominant_emotion']
    seconds = time.time()
    cv2.putText(frame,
              result,
              (50,50),
              font, 3,
              (0, 0, 255),
              2,
              cv2.LINE_4)
    cv2.imshow('Emotion recognition', frame)
    
    # In this section, we want to create a new audio file after each 3 seconds but only if the last prediction, that happened
    # 3 seconds before, is diffrent from the current predicton. We don't wanna here the same phrase two consecutive times.
    # So if you may here the audio file after 6 seconds or more..
    if result == last_result:
        diffrence_happend = True
    else:
        diffrence_happend = False
    
    if time.time() > last_time+3 and diffrence_happend == False:
        audio(result)
        last_time = time.time()
        last_result = result
    
    # Press q key if you wan't to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()